Within this notebook, I will be loading in and processing data from The Bureau of Investigative Journalism (https://www.thebureauinvestigates.com/) regarding US airstrike warfare in the Middle East. The ultimate target location for this data is within a tableau visualization.

In [1]:
#Library Import block
import pandas as pd
import csv
import glob

In [2]:
#Function to return the columns names within a csv file
def find_column_names(file):
    with open(file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        row_iter = 0
        for row in reader:
            if row_iter < 1:
                col_idx = 0
                for column in row:
                    print(col_idx, column)
                    col_idx += 1
                row_iter += 1
            else:
                break
            break

In [3]:
#Take average of the min and max numbers reported for kills and casualties
def take_average(min_num, max_num):
    return (min_num + max_num) / 2

In [4]:
file_list = [filename for filename in glob.glob('./data/*.csv')]

#Iterate through all files location within the data folder and return the column headings (index #)
for file in file_list:
    print('file name:', file)
    find_column_names(file)

file name: ./data\Afghanistan.csv
0 Strike ID
1 Date
2 Village/local area
3 District
4 Province
5 Type of attack
6 Reportedly drone?
7 US confirmed?
8 US only source?
9 Counter-
terrorism, force protection or strategic effects?
10 Reported target group?
11 Time
12 Reported target type
13 Minimum strikes
14 Maximum strikes
15 Minimum total people killed
16 Maximum total people killed
17 Minimum civilians reported killed
18 Maximum civilians reported killed
19 Minimum children reported killed
20 Maximum children reported killed
21 Minimum people injured
22 Maximum people injured
23 Strike link
24 Timeline URL
25 Index
file name: ./data\Pakistan.csv
0 Strike ID
1 Date
2 Location
3 Area
4 Minimum total people killed
5 Maximum total people killed
6 Minimum civilians reported killed
7 Maximum civilians reported killed
8 Minimum children reported killed
9 Maximum children reported killed
10 Minimum people injured
11 Maximum people injured
12 Strike link
13 
14 Index
file name: ./data\Somalia.

#The following are the column headings that I want to keep for my output
Strike ID
Date
Minimum total people killed
Maximum total people killed
Minimum civilians reported killed
Maximum civilians reported killed
Minimum people injured
Maximum people injured


In [5]:
#Create list containers for the columns from files I want to keep as my output
strike_id = []
date = []
country = []
minimum_total_people_killed = []
maximum_total_people_killed = []
minimum_civilians_reported_killed = []
maximum_civilians_reported_killed = []
minimum_people_injured = []
maximum_people_injured = []

In [6]:
#Now, for each file in my file list, make a list of the column index positions to gather for my analysis
#First, define a function to spit back the list
def get_file_indices(file):
    #0 and 1 are always included given file formatting
    col_idx_list = [0,1]
    with open(file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        row_iter = 0
        for row in reader:
            if row_iter < 1:
                col_idx_list.append(row.index("Minimum total people killed"))
                col_idx_list.append(row.index("Maximum total people killed"))
                col_idx_list.append(row.index("Minimum civilians reported killed"))
                col_idx_list.append(row.index("Maximum civilians reported killed"))
                col_idx_list.append(row.index("Minimum people injured"))
                col_idx_list.append(row.index("Maximum people injured"))
                row_iter += 1
            else:
                break
    #print(col_idx_list) #debugging line
    return col_idx_list


In [7]:
#This will build out all of my column lists from all the files
for file in file_list:
    columns = get_file_indices(file)
    #Now, we iterate through all rows (except header) and append them to list shells created above
    with open(file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        row_iter = 0
        for row in reader:
            if row_iter < 1:
                row_iter += 1
                continue
            else:
                country.append(file) #Need to parse out the actual name of the country here
                strike_id.append(row[columns[0]])
                date.append(row[columns[1]])
                minimum_total_people_killed.append(int(row[columns[2]].strip()))
                maximum_total_people_killed.append(int(row[columns[3]].strip()))
                minimum_civilians_reported_killed.append(int(row[columns[4]]))
                maximum_civilians_reported_killed.append(int(row[columns[5]]))
                minimum_people_injured.append(int(row[columns[6]]))
                maximum_people_injured.append(int(row[columns[7]]))

In [8]:
#Now, build out some additional lists that I would like for my dataframe
ave_total_killed = []
ave_civilians_killed = []
ave_injured = []

list_idx_pos = 0

while list_idx_pos < len(strike_id):
    ave_total_killed.append(take_average(minimum_total_people_killed[list_idx_pos], maximum_total_people_killed[list_idx_pos]))
    ave_civilians_killed.append(take_average(minimum_civilians_reported_killed[list_idx_pos], maximum_civilians_reported_killed[list_idx_pos]))
    ave_injured.append(take_average(minimum_people_injured[list_idx_pos], maximum_people_injured[list_idx_pos]))
    list_idx_pos += 1

In [9]:
#Build out pandas dataframe
df = pd.DataFrame(list(zip(strike_id, date, minimum_total_people_killed, maximum_total_people_killed, ave_total_killed,
                          minimum_civilians_reported_killed, maximum_civilians_reported_killed, ave_civilians_killed,
                          minimum_people_injured, maximum_people_injured, ave_injured)),
                 columns = ['Strike ID', 'Date', 'Total People Killed (Min)', 'Total People Killed (Max)', 'Total People Killed (Best Estimate)',
                           'Civilians Killed (Min)', 'Civilians Killed (Max)', 'Civilians Killed (Best Estimate)',
                           'Civilian Injuries (Min)', 'Civilian Injuries (Max)', 'Civilian Injuries (Best Estimate)'])

In [10]:
df

,Strike ID,Date,Total People Killed (Min),Total People Killed (Max),Total People Killed (Best Estimate),Civilians Killed (Min),Civilians Killed (Max),Civilians Killed (Best Estimate),Civilian Injuries (Min),Civilian Injuries (Max),Civilian Injuries (Best Estimate)
0,AFG0001,1/1/2015,0,0,0.0,0,0,0.0,0,0,0.0
1,AFG0002,3/1/2015,18,18,18.0,0,0,0.0,0,0,0.0
2,AFG0003,3/1/2015,7,7,7.0,0,0,0.0,0,0,0.0
3,AFG0004,6/1/2015,0,0,0.0,0,0,0.0,0,0,0.0
4,AFG0005,7/1/2015,3,3,3.0,0,0,0.0,0,0,0.0
5,AFG0006,8/1/2015,6,6,6.0,0,0,0.0,3,3,3.0
6,AFG0007,11/1/2015,8,8,8.0,0,0,0.0,3,3,3.0
7,AFG0008,16/01/2015,3,3,3.0,0,0,0.0,0,0,0.0
8,AFG0009,17/01/2015,0,0,0.0,0,0,0.0,0,0,0.0
9,AFG0010,19/01/2015,0,0,0.0,0,0,0.0,0,0,0.0
